In [203]:
import numba as nb
import numpy as np
# TODO do the timeit on the whole function that calls this as well
@nb.jit(nopython=True)
def fastloopNumba_l(Nx, Ny,intensityRefracted,intensityRefracted2,Dy,Dx):
    """
    Accelerated part of the refraction calculation
    Args:
        Nx (int): shape[0] of the intensity refracted.
        Ny (int): shape[1] of the intensity refracted.
        intensityRefracted (2d numpy array): intensity before propag.
        intensityRefracted2 (2d numpy array): intensity after propag.
        Dy (2d numpy array): Displacement along x (in voxel).
        Dx (2d numpy array): Displacement along y (in voxel).
        DxFloor (2d numpy array): floored displacement.
        DyFloor (2d numpy array): floored displacement.
    Returns:
        intensityRefracted2 (2d numpy array): intensity after propag.
    """
    for i in range(Nx):
        for j in range(Ny):
            Iij=intensityRefracted[i,j]
            Dxtmp=Dx[i,j]
            Dytmp=Dy[i,j]
            if Dxtmp==0 and Dytmp==0:
                intensityRefracted2[i,j]+=Iij
                continue
            inew=i
            jnew=j
            #Calculating displacement bigger than a pixel
            if abs(Dxtmp)>1:
                x = np.floor(Dxtmp)
                inew=np.int(i+x)
                Dxtmp=Dxtmp-x
            if abs(Dytmp)>1:
                y = np.floor(Dytmp)
                jnew=np.int(j+y)
                Dytmp=Dytmp-y
            #Calculating sub-pixel displacement
            if 0<=inew and inew<Nx:
                if 0<=jnew and jnew<Ny:
                    intensityRefracted2[inew,jnew]+=Iij*(1-abs(Dxtmp))*(1-abs(Dytmp))
                    if inew<Nx-1:
                        if Dxtmp>=0:
                            if jnew<Ny-1:
                                if Dytmp>=0:
                                    intensityRefracted2[inew+1,jnew]+=Iij*Dxtmp*(1-Dytmp)
                                    intensityRefracted2[inew+1,jnew+1]+=Iij*Dxtmp*Dytmp
                                    intensityRefracted2[inew,jnew+1]+=Iij*(1-Dxtmp)*Dytmp
                            if jnew:
                                if Dytmp<0:
                                    intensityRefracted2[inew+1,jnew]+=Iij*Dxtmp*(1-abs(Dytmp))
                                    intensityRefracted2[inew+1,jnew-1]+=Iij*Dxtmp*abs(Dytmp)
                                    intensityRefracted2[inew,jnew-1]+=Iij*(1-Dxtmp)*abs(Dytmp)
                    
                    if inew>0:
                        if Dxtmp<0:
                            if jnew<Ny-1:
                                if Dytmp>=0:
                                    intensityRefracted2[inew-1,jnew]+=Iij*abs(Dxtmp)*(1-Dytmp)
                                    intensityRefracted2[inew-1,jnew+1]+=Iij*abs(Dxtmp)*Dytmp
                                    intensityRefracted2[inew,jnew+1]+=Iij*(1-abs(Dxtmp))*Dytmp
                            if jnew:
                                if Dytmp<0:
                                    intensityRefracted2[inew-1,jnew]+=Iij*abs(Dxtmp)*(1-abs(Dytmp))
                                    intensityRefracted2[inew-1,jnew-1]+=Iij*Dxtmp*Dytmp
                                    intensityRefracted2[inew,jnew-1]+=Iij*(1-abs(Dxtmp))*abs(Dytmp)
    return intensityRefracted2

@nb.njit
def fastloopNumba(intensity_0, d_y, d_x):
    """
    Accelerated part of the refraction calculation

    Args:
        Nx (int): shape[0] of the intensity refracted.
        Ny (int): shape[1] of the intensity refracted.
        intensityRefracted (2d numpy array): intensity before propag.
        intensityRefracted2 (2d numpy array): intensity after propag.
        Dy (2d numpy array): Displacement along x (in voxel).
        Dx (2d numpy array): Displacement along y (in voxel).

    Returns:
        intensityRefracted2 (2d numpy array): intensity after propag.

    """
    n_y, n_x = intensity_0.shape
    intensity_z = np.zeros((n_y, n_x), dtype=np.float64)
    for i, i_i in enumerate(intensity_0):
        for j, i_ij in enumerate(i_i):
            dx_ij=d_x[i,j]
            dy_ij=d_y[i,j]
            if not dx_ij and not dy_ij:
                intensity_z[i,j]+=i_ij
                continue
            i_new=i
            j_new=j
            #Calculating displacement bigger than a pixel
            if np.abs(dx_ij)>1:
                x = np.floor(dx_ij)
                i_new=int(i+x)
                dx_ij=dx_ij-x
            if np.abs(dy_ij)>1:
                y = np.floor(dy_ij)
                j_new=int(j+y)
                dy_ij=dy_ij-y
            # Calculating sub-pixel displacement
            if 0<=i_new and i_new<n_y and 0<=j_new and j_new<n_x:
                intensity_z[i_new,j_new]+=i_ij*(1-np.abs(dx_ij))*(1-np.abs(dy_ij))
                if i_new<n_y-1 and dx_ij>=0:
                    if j_new<n_y-1 and dy_ij>=0:
                        intensity_z[i_new+1, j_new]+=i_ij*dx_ij*(1-dy_ij)
                        intensity_z[i_new+1, j_new+1]+=i_ij*dx_ij*dy_ij
                        intensity_z[i_new, j_new+1]+=i_ij*(1-dx_ij)*dy_ij
                    if j_new and dy_ij<0:
                        intensity_z[i_new+1,j_new]+=i_ij*dx_ij*(1-np.abs(dy_ij))
                        intensity_z[i_new+1,j_new-1]+=i_ij*dx_ij*np.abs(dy_ij)
                        intensity_z[i_new,j_new-1]+=i_ij*(1-dx_ij)*np.abs(dy_ij)
                if i_new and dx_ij<0:
                    if j_new<n_x-1 and dy_ij>=0:
                        intensity_z[i_new-1,j_new]+=i_ij*np.abs(dx_ij)*(1-dy_ij)
                        intensity_z[i_new-1,j_new+1]+=i_ij*np.abs(dx_ij)*dy_ij
                        intensity_z[i_new,j_new+1]+=i_ij*(1-np.abs(dx_ij))*dy_ij
                    if j_new and dy_ij<0:
                        intensity_z[i_new-1,j_new]+=i_ij*np.abs(dx_ij)*(1-np.abs(dy_ij))
                        intensity_z[i_new-1,j_new-1]+=i_ij*dx_ij*dy_ij
                        intensity_z[i_new,j_new-1]+=i_ij*(1-np.abs(dx_ij))*np.abs(dy_ij)
    return intensity_z

@nb.njit
def fastloopNumba2(intensity_0, d_y, d_x):
    """
    Accelerated part of the refraction calculation

    Args:
        Nx (int): shape[0] of the intensity refracted.
        Ny (int): shape[1] of the intensity refracted.
        intensityRefracted (2d numpy array): intensity before propag.
        intensityRefracted2 (2d numpy array): intensity after propag.
        Dy (2d numpy array): Displacement along x (in voxel).
        Dx (2d numpy array): Displacement along y (in voxel).

    Returns:
        intensityRefracted2 (2d numpy array): intensity after propag.

    """
    n_y, n_x = intensity_0.shape
    intensity_z = np.zeros((n_y, n_x), dtype=np.float64)
    for i, i_i in enumerate(intensity_0):
        for j, i_ij in enumerate(i_i):
            dx_ij=d_x[i,j]
            dy_ij=d_y[i,j]
            if not dx_ij and not dy_ij:
                intensity_z[i,j]+=i_ij
                continue
            i_new=i
            j_new=j
            #Calculating displacement bigger than a pixel
            if abs(dx_ij)>1:
                x = np.floor(dx_ij)
                i_new=int(i+x)
                dx_ij=dx_ij-x
            if abs(dy_ij)>1:
                y = np.floor(dy_ij)
                j_new=int(j+y)
                dy_ij=dy_ij-y
            # Calculating sub-pixel displacement
            if 0<=i_new and i_new<n_y and 0<=j_new and j_new<n_x:
                intensity_z[i_new,j_new]+=i_ij*(1-abs(dx_ij))*(1-abs(dy_ij))
                if i_new<n_y-1 and dx_ij>=0:
                    if j_new<n_y-1 and dy_ij>=0:
                        intensity_z[i_new+1, j_new]+=i_ij*dx_ij*(1-dy_ij)
                        intensity_z[i_new+1, j_new+1]+=i_ij*dx_ij*dy_ij
                        intensity_z[i_new, j_new+1]+=i_ij*(1-dx_ij)*dy_ij
                    if j_new and dy_ij<0:
                        intensity_z[i_new+1,j_new]+=i_ij*dx_ij*(1-abs(dy_ij))
                        intensity_z[i_new+1,j_new-1]+=i_ij*dx_ij*abs(dy_ij)
                        intensity_z[i_new,j_new-1]+=i_ij*(1-dx_ij)*abs(dy_ij)
                if i_new and dx_ij<0:
                    if j_new<n_x-1 and dy_ij>=0:
                        intensity_z[i_new-1,j_new]+=i_ij*abs(dx_ij)*(1-dy_ij)
                        intensity_z[i_new-1,j_new+1]+=i_ij*abs(dx_ij)*dy_ij
                        intensity_z[i_new,j_new+1]+=i_ij*(1-abs(dx_ij))*dy_ij
                    if j_new and dy_ij<0:
                        intensity_z[i_new-1,j_new]+=i_ij*abs(dx_ij)*(1-abs(dy_ij))
                        intensity_z[i_new-1,j_new-1]+=i_ij*dx_ij*dy_ij
                        intensity_z[i_new,j_new-1]+=i_ij*(1-abs(dx_ij))*abs(dy_ij)
    return intensity_z

#TODO this is the fastest if the data is generated in a stack
@nb.njit
def fastloopNumba3(idxdy):
    """
    Accelerated part of the refraction calculation

    Args:
        Nx (int): shape[0] of the intensity refracted.
        Ny (int): shape[1] of the intensity refracted.
        intensityRefracted (2d numpy array): intensity before propag.
        intensityRefracted2 (2d numpy array): intensity after propag.
        Dy (2d numpy array): Displacement along x (in voxel).
        Dx (2d numpy array): Displacement along y (in voxel).

    Returns:
        intensityRefracted2 (2d numpy array): intensity after propag.

    """
    n_y, _, n_x = idxdy.shape
    intensity_z = np.zeros((n_y, n_x), dtype=np.float64)
    for i, (i_i, dx_i, dy_i) in enumerate(idxdy):
        for j, (i_ij, dx_ij, dy_ij) in enumerate(zip(i_i, dx_i, dy_i)):
            # dx_ij=d_x[i,j]
            # dy_ij=d_y[i,j]
            if not dx_ij and not dy_ij:
                intensity_z[i,j]+=i_ij
                continue
            i_new=i
            j_new=j
            #Calculating displacement bigger than a pixel
            if abs(dx_ij)>1:
                x = np.floor(dx_ij)
                i_new=int(i+x)
                dx_ij=dx_ij-x
            if abs(dy_ij)>1:
                y = np.floor(dy_ij)
                j_new=int(j+y)
                dy_ij=dy_ij-y
            # Calculating sub-pixel displacement
            if 0<=i_new and i_new<n_y and 0<=j_new and j_new<n_x:
                intensity_z[i_new,j_new]+=i_ij*(1-abs(dx_ij))*(1-abs(dy_ij))
                if i_new<n_y-1 and dx_ij>=0:
                    if j_new<n_y-1 and dy_ij>=0:
                        intensity_z[i_new+1, j_new]+=i_ij*dx_ij*(1-dy_ij)
                        intensity_z[i_new+1, j_new+1]+=i_ij*dx_ij*dy_ij
                        intensity_z[i_new, j_new+1]+=i_ij*(1-dx_ij)*dy_ij
                    if j_new and dy_ij<0:
                        intensity_z[i_new+1,j_new]+=i_ij*dx_ij*(1-abs(dy_ij))
                        intensity_z[i_new+1,j_new-1]+=i_ij*dx_ij*abs(dy_ij)
                        intensity_z[i_new,j_new-1]+=i_ij*(1-dx_ij)*abs(dy_ij)
                if i_new and dx_ij<0:
                    if j_new<n_x-1 and dy_ij>=0:
                        intensity_z[i_new-1,j_new]+=i_ij*abs(dx_ij)*(1-dy_ij)
                        intensity_z[i_new-1,j_new+1]+=i_ij*abs(dx_ij)*dy_ij
                        intensity_z[i_new,j_new+1]+=i_ij*(1-abs(dx_ij))*dy_ij
                    if j_new and dy_ij<0:
                        intensity_z[i_new-1,j_new]+=i_ij*abs(dx_ij)*(1-abs(dy_ij))
                        intensity_z[i_new-1,j_new-1]+=i_ij*dx_ij*dy_ij
                        intensity_z[i_new,j_new-1]+=i_ij*(1-abs(dx_ij))*abs(dy_ij)
    return intensity_z


@nb.njit
def fastloopNumba4(idxdy):
    """
    Accelerated part of the refraction calculation

    Args:
        Nx (int): shape[0] of the intensity refracted.
        Ny (int): shape[1] of the intensity refracted.
        intensityRefracted (2d numpy array): intensity before propag.
        intensityRefracted2 (2d numpy array): intensity after propag.
        Dy (2d numpy array): Displacement along x (in voxel).
        Dx (2d numpy array): Displacement along y (in voxel).

    Returns:
        intensityRefracted2 (2d numpy array): intensity after propag.

    """
    n_y, n_x, _ = idxdy.shape
    intensity_z = np.zeros((n_y, n_x), dtype=np.float64)
    for i, k in enumerate(idxdy):
        for j, m in enumerate(k):
            i_ij, dx_ij, dy_ij = m
            if not dx_ij and not dy_ij:
                intensity_z[i,j]+=i_ij
                continue
            i_new=i
            j_new=j
            #Calculating displacement bigger than a pixel
            if abs(dx_ij)>1:
                x = np.floor(dx_ij)
                i_new=int(i+x)
                dx_ij=dx_ij-x
            if abs(dy_ij)>1:
                y = np.floor(dy_ij)
                j_new=int(j+y)
                dy_ij=dy_ij-y
            # Calculating sub-pixel displacement
            if 0<=i_new and i_new<n_y and 0<=j_new and j_new<n_x:
                intensity_z[i_new,j_new]+=i_ij*(1-abs(dx_ij))*(1-abs(dy_ij))
                if i_new<n_y-1 and dx_ij>=0:
                    if j_new<n_y-1 and dy_ij>=0:
                        intensity_z[i_new+1, j_new]+=i_ij*dx_ij*(1-dy_ij)
                        intensity_z[i_new+1, j_new+1]+=i_ij*dx_ij*dy_ij
                        intensity_z[i_new, j_new+1]+=i_ij*(1-dx_ij)*dy_ij
                    if j_new and dy_ij<0:
                        intensity_z[i_new+1,j_new]+=i_ij*dx_ij*(1-abs(dy_ij))
                        intensity_z[i_new+1,j_new-1]+=i_ij*dx_ij*abs(dy_ij)
                        intensity_z[i_new,j_new-1]+=i_ij*(1-dx_ij)*abs(dy_ij)
                if i_new and dx_ij<0:
                    if j_new<n_x-1 and dy_ij>=0:
                        intensity_z[i_new-1,j_new]+=i_ij*abs(dx_ij)*(1-dy_ij)
                        intensity_z[i_new-1,j_new+1]+=i_ij*abs(dx_ij)*dy_ij
                        intensity_z[i_new,j_new+1]+=i_ij*(1-abs(dx_ij))*dy_ij
                    if j_new and dy_ij<0:
                        intensity_z[i_new-1,j_new]+=i_ij*abs(dx_ij)*(1-abs(dy_ij))
                        intensity_z[i_new-1,j_new-1]+=i_ij*dx_ij*dy_ij
                        intensity_z[i_new,j_new-1]+=i_ij*(1-abs(dx_ij))*abs(dy_ij)
    return intensity_z
import numpy as np
import matplotlib.pyplot as plt
import timeit
rng = np.random.default_rng()
size = (2000, 2000)
margin = 10
test_data = np.pad(10000*rng.random(size=size), margin)
dx = np.pad(10*(rng.random(size=size)-0.5), margin)
dy = np.pad(10*(rng.random(size=size)-0.5), margin)
zipped = zip(test_data, dy, dx)
size2 = [i+2*margin for i in size]
data = np.stack([test_data, dx, dy], axis=1)
data2 = np.stack([test_data, dx, dy], axis=2)
%timeit fastloopNumba_l(*size2, test_data, np.zeros(size2), dy, dx)
%timeit fastloopNumba(test_data, dy, dx)
%timeit fastloopNumba2(test_data, dy, dx)
%timeit fastloopNumba3(data)
%timeit fastloopNumba4(data2)
%timeit fastloopNumba_l(*size2, test_data, np.zeros(size2), dy, dx)
%timeit fastloopNumba(test_data, dy, dx)
%timeit fastloopNumba2(test_data, dy, dx)
%timeit fastloopNumba3(data)
%timeit fastloopNumba4(data2)
print((fastloopNumba_l(*size2, test_data, np.zeros(size2), dy, dx) == fastloopNumba(test_data, dy, dx)).all())
print((fastloopNumba_l(*size2, test_data, np.zeros(size2), dy, dx) == fastloopNumba2(test_data, dy, dx)).all())
print((fastloopNumba_l(*size2, test_data, np.zeros(size2), dy, dx) == fastloopNumba3(np.stack([test_data, dx, dy], axis=1))).all())
print((fastloopNumba_l(*size2, test_data, np.zeros(size2), dy, dx) == fastloopNumba4(np.stack([test_data, dx, dy], axis=2))).all())
print(np.sqrt(np.max((fastloopNumba_l(*size2, test_data, np.zeros(size2), dy, dx) - fastloopNumba(test_data, dy, dx))**2)))
print(np.sqrt(np.max((fastloopNumba_l(*size2, test_data, np.zeros(size2), dy, dx) - fastloopNumba2(test_data, dy, dx))**2)))
print(np.sqrt(np.max((fastloopNumba_l(*size2, test_data, np.zeros(size2), dy, dx) - fastloopNumba3(np.stack([test_data, dx, dy], axis=1)))**2)))
print(np.sqrt(np.max((fastloopNumba_l(*size2, test_data, np.zeros(size2), dy, dx) - fastloopNumba4(np.stack([test_data, dx, dy], axis=2)))**2)))

# print(np.sum(fastloopNumba_l(*size2, test_data, np.zeros(size2), dy, dx)))
# print(np.sum(test_data))
# print(np.sum(fastloopNumba2(test_data, dy, dx)))

# size = (5,5)
# test_data = np.ones(size)
# dx = np.zeros(size)
# dy = np.zeros(size)
# dx[2,2] = 2

# fig, axs = plt.subplots(3,2, figsize=(18,18))
# fig.colorbar(axs[0][0].imshow(test_data), ax=axs[0][0])
# fig.colorbar(axs[0][1].imshow(fastloopNumba(test_data, dy, dx)), ax=axs[0][1])
# fig.colorbar(axs[1][0].imshow(dx), ax=axs[1][0])
# fig.colorbar(axs[1][1].imshow(dy), ax=axs[1][1])
# fig.colorbar(axs[2][0].imshow(fastloopNumba(test_data, dy, dx)-fastloopNumba_l(*size2, test_data, np.zeros(size2), dy, dx)), ax=axs[2][0])
# fig.tight_layout()
# plt.show()

41.5 ms ± 457 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
36.2 ms ± 70.9 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
36.6 ms ± 72 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
33.9 ms ± 128 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
39.4 ms ± 217 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
41.2 ms ± 505 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
36.5 ms ± 302 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
36.8 ms ± 306 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
33.9 ms ± 332 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
39.5 ms ± 424 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
True
True
True
True
0.0
0.0
0.0
0.0


Probably used small angle approximations

In [ ]:
import numpy as np
test_data = np.pad(10000*rng.random(size=size), margin)
dx = np.pad(10*(rng.random(size=size)-0.5), margin)
# @nb.njit
# def test(a = np.stack([test_data, dx], axis=1)):
#     for i, (i_i, dx_i) in enumerate(a):
#         for j, (i_ij, dx_ij) in enumerate()
#         print(i, j, k)
# # test()
np.stack([test_data, dx], axis=1).shape

In [ ]:
import cv2
idk_x, idk_y = np.meshgrid(np.arange(5), np.arange(5))
cv2.remap(test_data, idk_x, idk_y, interpolation= cv2.INTER_LINEAR)

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
SIZE = (3,3)
img = np.random.rand(*SIZE)*255
map_y = np.random.random(size=SIZE).astype(np.float32)
map_x = np.random.random(size=SIZE).astype(np.float32)
# img = test_data
map_x = dx.astype(np.float32)
# map_y = dy.astype(np.float32)
# map_x = np.zeros((5,5)).astype(np.float32)
# # map_x, map_y = np.meshgrid(np.arange(0, 5), np.arange(0, 5))
map_x = np.ones(SIZE).astype(np.float32)
map_y = np.ones(SIZE).astype(np.float32)
map_x[1][1]=0
map_y[1][1]=0
# map_y = np.float32(map_y)
# map_y = np.array([[0, 1,2], [2, 3,3],[2, 3,3]], dtype=np.float32)
# map_x = np.array([[0, 1,2], [7, 10,7],[7, 10,7]], dtype=np.float32)
mapped_img = cv2.remap(img, map_x, map_y, cv2.INTER_LINEAR)
fig, axs = plt.subplots(2,2)
fig.colorbar(axs[0][0].imshow(img), ax=axs[0][0])
fig.colorbar(axs[0][1].imshow(mapped_img), ax=axs[0][1])
fig.colorbar(axs[1][0].imshow(map_x), ax=axs[1][0])
fig.colorbar(axs[1][1].imshow(map_y), ax=axs[1][1])
plt.show()
print(np.max(mapped_img),np.min(mapped_img), np.max(img), img[0][0], img[1][1])


In [ ]:
import numpy as np
import pyopencl as cl

a_np = np.random.rand(50000).astype(np.float32)
b_np = np.random.rand(50000).astype(np.float32)

ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx)

mf = cl.mem_flags
a_g = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=a_np)
b_g = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=b_np)

prg = cl.Program(ctx, """
__kernel void sum(
    __global const float *a_g, __global const float *b_g, __global float *res_g)
{
  int gid = get_global_id(0);
  res_g[gid] = a_g[gid] + b_g[gid];
}
""").build()

res_g = cl.Buffer(ctx, mf.WRITE_ONLY, a_np.nbytes)
knl = prg.sum  # Use this Kernel object for repeated calls
knl(queue, a_np.shape, None, a_g, b_g, res_g)

res_np = np.empty_like(a_np)
cl.enqueue_copy(queue, res_np, res_g)

# Check on CPU with Numpy:
print(res_np - (a_np + b_np))
print(np.linalg.norm(res_np - (a_np + b_np)))
assert np.allclose(res_np, a_np + b_np)

In [ ]:
import pyopencl as cl
import pyopencl.array as cl_array
import numpy
import numpy.linalg as la

a = numpy.random.rand(50000).astype(numpy.float32)
b = numpy.random.rand(50000).astype(numpy.float32)

ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx)

a_dev = cl_array.to_device(queue, a)
b_dev = cl_array.to_device(queue, b)
dest_dev = cl_array.empty_like(a_dev)

prg = cl.Program(ctx, """
    __kernel void sum(__global const float *a,
    __global const float *b, __global float *c)
    {
      int gid = get_global_id(0);
      c[gid] = a[gid] + b[gid];
    }
    """).build()

knl = prg.sum  # Use this Kernel object for repeated calls
knl(queue, a.shape, None, a_dev.data, b_dev.data, dest_dev.data)

print(la.norm((dest_dev - (a_dev+b_dev)).get()))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import timeit
def cylinder1(radius, length, pixel_size):
    radius = round(radius/pixel_size)
    coordinates = np.broadcast_to(np.arange(-radius, radius+1),
                                    (int(-(length//-1)), 2*radius+1))
    cylinder = (radius/pixel_size)**2 - coordinates**2
    cylinder[cylinder < 0] = 0
    cylinder = 2*np.sqrt(cylinder)
    return cylinder

def cylinder2(radius, length, pixel_size):
    radius = round(radius/pixel_size)
    coordinates = np.arange(-radius, radius+1)
    cylinder = (radius/pixel_size)**2 - coordinates**2
    cylinder = np.outer(np.ones(length), cylinder)
    cylinder[cylinder < 0] = 0
    cylinder = 2*np.sqrt(cylinder)
    return cylinder

def cylinder3(radius, length, pixel_size):
    radius = round(radius/pixel_size)
    coordinates = np.arange(-radius, radius+1)
    cylinder = (radius/pixel_size)**2 - coordinates**2
    cylinder = np.add.outer(np.zeros(length), cylinder)
    cylinder[cylinder < 0] = 0
    cylinder = 2*np.sqrt(cylinder)
    return cylinder


args = [50, 100, 1]
%timeit cylinder1(*args)
%timeit cylinder2(*args)
%timeit cylinder3(*args)
plt.figure()
plt.imshow(cylinder1(*args))
plt.colorbar()
plt.figure()
plt.imshow(cylinder2(*args))
plt.colorbar()
plt.figure()
plt.imshow(cylinder3(*args, 200, 200))
plt.colorbar()
plt.figure()
plt.imshow(cylinder1(*args)-cylinder2(*args))
plt.colorbar()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import timeit

def torus1(minor_radius, major_radius, pixel_size)  :      
        minor_radius = round(minor_radius/pixel_size)
        major_radius = round(major_radius/pixel_size)
        coordinates = np.meshgrid(np.arange(-major_radius-minor_radius, major_radius+minor_radius+1),
                                  np.arange(-major_radius-minor_radius, major_radius+minor_radius+1))
        sqrt = np.sqrt(np.sum([i**2 for i in coordinates], axis=0))
        torus = 2*np.sqrt((minor_radius/pixel_size)**2 -
                          (sqrt-major_radius/pixel_size)**2)
        torus[np.isnan(torus)] = 0
        return torus

def torus2(minor_radius, major_radius, pixel_size)  :      
        minor_radius = round(minor_radius/pixel_size)
        major_radius = round(major_radius/pixel_size)
        coords = np.arange(-major_radius-minor_radius, major_radius+minor_radius+1)
        coords **= 2
        coords = np.sqrt(np.add.outer(coords, coords))
        coords -= major_radius/pixel_size
        # coords[coords < 0] = 0
        torus = (minor_radius/pixel_size)**2-coords**2
        torus[torus < 0] = 0
        torus = 2*np.sqrt(torus)
        # torus[np.isnan(torus)] = 0
        return torus
args = [100, 200, 1]
%timeit torus1(*args)
%timeit torus2(*args)
plt.figure()
plt.imshow(torus1(*args))
plt.colorbar()
plt.figure()
plt.imshow(torus2(*args))
plt.colorbar()
plt.figure()
plt.imshow(torus2(*args)-torus1(*args))
plt.colorbar()

In [ ]:
import itertools
from scipy.constants import golden_ratio
def regular(self, radius):
    self.radius = radius

    meh = np.arange(0, 720, 2)
    idk = self.radius*np.sqrt(meh)
    idk2 = (np.pi*meh/golden_ratio) % (2*np.pi)
    x = idk*np.cos(idk2)
    y = idk*np.sin(idk2)
    return zip(x, y)

def regular2(self, radius):
    self.radius = radius

    meh = np.arange(0, 720, 2)
    idk = self.radius*np.sqrt(meh)
    idk2 = (np.pi*meh/golden_ratio) % (2*np.pi)
    x = idk*np.cos(idk2)
    y = idk*np.sin(idk2)
    return itertools.zip_longest(x,y)

# def regular2(self, dimensions, spacing):
#     self.dimensions = dimensions
#     self.grid_spacing = spacing


#     x_coords = np.linspace(-(self.dimensions[0]//2),
#                             -(self.dimensions[0]//-2),
#                             int(self.dimensions[0]/self.grid_spacing)+1)
#     y_coords = np.linspace(-(self.dimensions[1]//2),
#                             -(self.dimensions[1]//-2),
#                             int(self.dimensions[1]/self.grid_spacing)+1)
#     return ((i,j)  for j in y_coords for i in x_coords)

# def regular3(self, dimensions, spacing):
#     self.dimensions = dimensions
#     self.grid_spacing = spacing


#     x_coords = np.linspace(-(self.dimensions[0]//2),
#                             -(self.dimensions[0]//-2),
#                             int(self.dimensions[0]/self.grid_spacing)+1)
#     y_coords = np.linspace(-(self.dimensions[1]//2),
#                             -(self.dimensions[1]//-2),
#                             int(self.dimensions[1]/self.grid_spacing)+1)
#     return itertools.product(x_coords, y_coords)
    
import numpy as np
import timeit
args = [1]
slef = type('regular', (), {})
from collections import Counter
# print(list(regular(slef, *args)))
# print(list(regular2(slef, *args)))
%timeit list(regular(slef,*args))
%timeit list(regular2(slef,*args))
# %timeit list(regular3(slef,*args))
list(regular2(slef,*args))[1] == list(regular(slef,*args))[1]
